# Frameworks

Como hemos visto cada vez que hemos creado una nueva arquitectura, ya sea un solo perceptron, varios, con o sin funciones de activación, hemos tenido que hacer todos los cálculos de derivadas y programarlos.

Esto no es nada eficiente, ya que se pierde un montón de tiempo, da lugar a bugs que son muy difíciles de depurar y el código no está todo lo optimizado que debería.

Por ello es necesario el uso de frameworks que nos evitan esto.

Ahora mismo hay dos frameworks por encime del resto [Pytorch](https://pytorch.org/) y [TensoFlow](https://www.tensorflow.org/). En ambos la implementación de redes neuronales y su entrenamiento se hace con unas pocas líneas de código y no tenemos que ser nosotros los que tengamos que codificar todo.

Vamos a ver dos ejemplos donde se hace esto

## **Pytorch**

``` python
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
from torch import nn
import torch


training_data = datasets.CIFAR10(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)
test_data = datasets.CIFAR10(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)


BS = 64
train_dataloader = DataLoader(training_data, batch_size=BS, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=BS, shuffle=True)


class NeuralNetworkFromScratch(nn.Module):
    def __init__(self):
        super(NeuralNetworkFromScratch, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(3*32*32, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
model_scratch = NeuralNetworkFromScratch()


device = "cuda" if torch.cuda.is_available() else "cpu"
model_scratch.to(device)


LR = 1e-2
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model_scratch.parameters(), lr=LR)


def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # X and y to device
        X, y = X.to(device), y.to(device)

        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            # X and y to device
            X, y = X.to(device), y.to(device)
            
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model_scratch, loss_fn, optimizer)
    test_loop(test_dataloader, model_scratch, loss_fn)
print("Done!")
```

## **TensorFlow**

``` python
import tensorflow as tf


mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0


model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


model.fit(x_train, y_train, epochs=5)
model.evaluate(x_test,  y_test, verbose=2)
```

Como se puede ver con Pytorch se necesita mucho más código que con TensorFlow, esto es porque en realidad TensorFlow adoptó a Keras como una librería de alto nivel que actúa por encima y que reduce mucho el código. Por lo que para comparar realmente la cantidad de código deberíamos comparar una librería de más alto nivel sobre Pytorch con TensorFlow usando Keras. Sin embargo no hay una librería oficial por encima de Pytorch, las dos más populares son [fastai](https://www.fast.ai/) o [Pytorch Lightning](https://www.pytorchlightning.ai/).

Vamos a ver un ejemplo sencillo con fastai

## **FastAI**

```python
from fastai.vision.all import *
path = untar_data(URLs.PETS)/'images'

def is_cat(x): return x[0].isupper()
dls = ImageDataLoaders.from_name_func(
    path, get_image_files(path), valid_pct=0.2, seed=42,
    label_func=is_cat, item_tfms=Resize(224))

learn = cnn_learner(dls, resnet34, metrics=error_rate)
learn.fine_tune(1)
```

Como se puede ver, usando una librería por encima de Pytorch el código se reduce

## ¿Por qué elegir Pytorch frente a TensorFlow?

Pytorch tiene muchas ventajas frente a tensorFlow. Como hemos visto antes no podemos compararlos por la cantidad de código que se genera con uno y con otro, porque vemos que no están a la misma altura. Si usamos una librería de alto nivel con Pytorch, la cantidad de lineas que necesitamos baja mucho

### Facilidad de uso

Pytorch se diseño para programarse muy parecido a como se programa en Python, sobre todo cómo se programa Python usando [Numpy](https://numpy.org/), que es una librería de cálculo matricial muy usada dentro del ecosistema Python. Por lo que si sabes usar Numpy, aprender Pytorch te será muy sencillo, ya que incluso muchas funciones se llaman igual, con la diferencia de que con Pytorch podemos hacer las operaciones en la GPU, lo cual se hace mucho más rápido

### Disponibilidad de redes preentrenadas

Como ya se ha explicado hay redes como GPT-3 que tienen tantos parámetros que son imposibles de entrenar desde cero por cualquier persona, incluso empresa, por lo que poder utilizar modelos preentrenados de redes neuronales y ajustarlas a nuestro problema es esencial

[HuggingFace](https://huggingface.co/) es una de las páginas más usadas para descargarse estos modelos preentrenados. En la siguiente imagen podemos ver como hay una gran cantidad de modelos disponibles solo para Pytorch, mientras que para TensorFlow o ambos son muchos menos. Por lo que a la hora de poderse descargar redes preentrenadas es mejor si estamos utilizando Pytorch.

![Number-of-Models-on-HuggingFace](Imagenes/Number-of-Models-on-HuggingFace.png)

Si analizamos los 30 modelos más populares podemos ver que todos están disponibles para Pytorch, mientras que solo unos dos tercios lo están para TensorFlow

![Number-of-Top-30-Models-on-HuggingFace](Imagenes/Number-of-Top-30-Models-on-HuggingFace.png)

### Investigación

Esta es una especialidad muy nueva, por lo que constantemente están surgiendo nuevos modelos. Implementarlos y entrenarlos desde cero supone una grán pérdida de tiempo, por lo que es mejor si podemos clonar un repositorio y usarlo. En la siguiente gráfica podemos ver el porcentaje de papers publicados con Pytorch y TensorFlow

![Fraction-of-Papers-Using-PyTorch-vs.-TensorFlow](Imagenes/Fraction-of-Papers-Using-PyTorch-vs.-TensorFlow.png)

Podemos ver como en pocos años Pytorch ha pasado de copar en torno al 7 % a que la mayoría de papers se publiquen con este framework

En la siguiente imagen podemos ver el trasvase de investigadores de un framweork a otro

![Sankey](Imagenes/Sankey.png)

Podemos ver como la mitad de los investigadores han pasado de TensorFlow a Pytorch, mientras que al revés casi no sucede. Y esta imagen es el paso entre 2018 y 2019, ahora probablemente sea más favorable aun hacia Pytorch

[Papers with code](https://paperswithcode.com/) es una página también muy famosa donde podemos encontrar códigos de los últimos papers sobre deep learning con código, hecho por los propios investigadores y/o por otras personas. Podemos ver el porcentaje de repositorios hechos con Pytorch, TensorFlow u otro framework, y vemos como Pytorch ha evolucionado quedándose con un gran porcentaje, mientras que TensorFlow ha ido dismunuyendo

![Percentage-of-Repositories-by-Framework](Imagenes/Percentage-of-Repositories-by-Framework.png)

### Implementación

En aplicaciones a nivel industrial TensorFlow está más extendido, ya que este framework, desde sus inicios dio más herramientas para la implementación y despliegue. Pero Pytorch hace tiempo que se puso las pilas y creo las suyas, por lo que no se queda atrás con respecto a TensorFlow. A nivel personal me he encontrado pocas empresas que trabajen con TensorFlow, la mayoría lo hace con Pytorch

Si entras en una empresa y trabajas con código ya hecho, es posible que te lo encuentre hecho con TensorFlow, pero para nuevas implementaciones es mejor Pytorch, ya que para nuevas implementaciones es mejor usar modelos más nuevos preentrenados, que como hemos visto, son más accesibles desde Pytorch. Al igual que antes, cada vez que quiero usar una red neuronal nueva, que tiene repositorio en GitHub, está implementada en Pytorch